In [58]:
import pandas as pd

In [59]:
latestmuni = pd.read_csv("2015muni/ACS_15_5YR_S0801_with_ann.csv", usecols=["GEO.id", "GEO.id2", "GEO.display-label", "HC01_EST_VC55", "HC01_MOE_VC55"])
prevmuni = pd.read_csv("2009muni/ACS_09_5YR_S0801_with_ann.csv", usecols=["GEO.id", "GEO.id2", "GEO.display-label", "HC01_EST_VC50", "HC01_MOE_VC50"])
prevcounty=pd.read_csv("2009county/ACS_09_5YR_S0801_with_ann.csv", usecols=["GEO.id", "GEO.id2", "GEO.display-label", "HC01_EST_VC50", "HC01_MOE_VC50"])
latestcounty=pd.read_csv("2015county/ACS_15_5YR_S0801_with_ann.csv", usecols=["GEO.id", "GEO.id2", "GEO.display-label", "HC01_EST_VC55", "HC01_MOE_VC55"])


In [60]:
# Drop added row
latestmuni = latestmuni[latestmuni['GEO.id'] != 'Id']
prevmuni = prevmuni[prevmuni['GEO.id'] != 'Id']
latestcounty = latestcounty[latestcounty['GEO.id'] != 'Id']
prevcounty = prevcounty[prevcounty['GEO.id'] != 'Id']

In [61]:
latestframes= [latestmuni, latestcounty]
prevframes=[prevmuni, prevcounty]
latestmuni = pd.concat(latestframes, ignore_index=True)
prevmuni = pd.concat(prevframes, ignore_index=True)

In [62]:
prevmuni.head()

,GEO.id,GEO.id2,GEO.display-label,HC01_EST_VC50,HC01_MOE_VC50
0,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",N,N
1,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",19.3,1.6
2,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",20.8,1.1
3,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",23.5,2.5
4,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",28.9,3.3


In [63]:
latestmuni = latestmuni.rename(columns={ # yes, we could have done this when we read in the CSV
        'HC01_EST_VC55': '2015meantime',
        'HC01_MOE_VC55': '2015meanmoe',
    })
prevmuni = prevmuni.rename(columns={ # yes, we could have done this when we read in the CSV
        'HC01_EST_VC50': '2009meantime',
        'HC01_MOE_VC50': '2009meanmoe',
    })


In [64]:
# Remove "new jersey" label
latestmuni['GEOdisplaylabel'] = latestmuni['GEO.display-label'].replace(to_replace=', New Jersey', value='', regex=True)
prevmuni['GEOdisplaylabel'] = prevmuni['GEO.display-label'].replace(to_replace=', New Jersey', value='', regex=True)

In [65]:
prevmuni.head()

,GEO.id,GEO.id2,GEO.display-label,2009meantime,2009meanmoe,GEOdisplaylabel
0,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",N,N,"County subdivisions not defined, Atlantic County"
1,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",19.3,1.6,"Absecon city, Atlantic County"
2,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",20.8,1.1,"Atlantic City city, Atlantic County"
3,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",23.5,2.5,"Brigantine city, Atlantic County"
4,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",28.9,3.3,"Buena borough, Atlantic County"


In [66]:
# Split town and county names
latest_newcols = pd.DataFrame(latestmuni.GEOdisplaylabel.str.split(',',1).tolist(), columns = ['GEOdisplaylabel', 'County'])
prev_newcols = pd.DataFrame(prevmuni.GEOdisplaylabel.str.split(',',1).tolist(), columns = ['GEOdisplaylabel', 'County'])

In [67]:
latestmuni = pd.concat([latest_newcols, latestmuni], axis=1)
prevmuni = pd.concat([prev_newcols, prevmuni], axis=1)

In [68]:
# Remove county label
latestmuni['County'] = latestmuni['County'].replace(to_replace='County', value='', regex=True)
prevmuni['County'] = prevmuni['County'].replace(to_replace='County', value='', regex=True)


In [69]:
muni_merge = pd.merge(latestmuni, prevmuni, left_on='GEO.id2', right_on='GEO.id2', how='left')

In [70]:
muni_merge.head()

,GEOdisplaylabel_x,County_x,GEO.id_x,GEO.id2,GEO.display-label_x,2015meantime,2015meanmoe,GEOdisplaylabel_x,GEOdisplaylabel_y,County_y,GEO.id_y,GEO.display-label_y,2009meantime,2009meanmoe,GEOdisplaylabel_y
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",County subdivisions not defined,Atlantic,0600000US3400100000,"County subdivisions not defined, Atlantic Coun...",N,N,"County subdivisions not defined, Atlantic County"
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",22.7,2.4,"Absecon city, Atlantic County",Absecon city,Atlantic,0600000US3400100100,"Absecon city, Atlantic County, New Jersey",19.3,1.6,"Absecon city, Atlantic County"
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",19.1,0.9,"Atlantic City city, Atlantic County",Atlantic City city,Atlantic,0600000US3400102080,"Atlantic City city, Atlantic County, New Jersey",20.8,1.1,"Atlantic City city, Atlantic County"
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",28.9,3.1,"Brigantine city, Atlantic County",Brigantine city,Atlantic,0600000US3400107810,"Brigantine city, Atlantic County, New Jersey",23.5,2.5,"Brigantine city, Atlantic County"
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",27.7,3.2,"Buena borough, Atlantic County",Buena borough,Atlantic,0600000US3400108680,"Buena borough, Atlantic County, New Jersey",28.9,3.3,"Buena borough, Atlantic County"


In [71]:
muni_merge = muni_merge.drop(["GEOdisplaylabel_y",'GEO.display-label_y', 'County_y', 'GEO.id_y'], axis=1)
muni_merge.rename(columns={'GEOdisplaylabel_x': 'GEOdisplaylabel', 'County_x': 'County', 'GEO.id_x': 'GEO.id', 'GEO.display-label_x': 'GEOdisplaylabel_long'}, inplace=True)

In [72]:
muni_merge.dtypes

GEOdisplaylabel         object
County                  object
GEO.id                  object
GEO.id2                 object
GEOdisplaylabel_long    object
2015meantime            object
2015meanmoe             object
GEOdisplaylabel         object
2009meantime            object
2009meanmoe             object
dtype: object

In [73]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015meantime,2015meanmoe,GEOdisplaylabel,2009meantime,2009meanmoe
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",N,N
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",22.7,2.4,"Absecon city, Atlantic County",19.3,1.6
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",19.1,0.9,"Atlantic City city, Atlantic County",20.8,1.1
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",28.9,3.1,"Brigantine city, Atlantic County",23.5,2.5
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",27.7,3.2,"Buena borough, Atlantic County",28.9,3.3


In [74]:
muni_merge["2015meanmoeper"] = pd.to_numeric(muni_merge["2015meanmoe"],errors="coerce")/ pd.to_numeric(muni_merge["2015meantime"], errors="coerce")
muni_merge["2009meanmoeper"] = pd.to_numeric(muni_merge["2009meanmoe"],errors="coerce") / pd.to_numeric(muni_merge["2009meantime"], errors="coerce")

In [75]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015meantime,2015meanmoe,GEOdisplaylabel,2009meantime,2009meanmoe,2015meanmoeper,2009meanmoeper
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",N,N,NaN,NaN
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",22.7,2.4,"Absecon city, Atlantic County",19.3,1.6,0.105727,0.082902
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",19.1,0.9,"Atlantic City city, Atlantic County",20.8,1.1,0.047120,0.052885
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",28.9,3.1,"Brigantine city, Atlantic County",23.5,2.5,0.107266,0.106383
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",27.7,3.2,"Buena borough, Atlantic County",28.9,3.3,0.115523,0.114187


In [76]:
muni_merge["nj_latest"] = 31
muni_merge["nj_previous"] = 29.6
muni_merge["us_latest"] = 25.9
muni_merge["us_previous"] = 25.2

In [77]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015meantime,2015meanmoe,GEOdisplaylabel,2009meantime,2009meanmoe,2015meanmoeper,2009meanmoeper,nj_latest,nj_previous,us_latest,us_previous
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",N,N,NaN,NaN,31,29.6,25.9,25.2
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",22.7,2.4,"Absecon city, Atlantic County",19.3,1.6,0.105727,0.082902,31,29.6,25.9,25.2
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",19.1,0.9,"Atlantic City city, Atlantic County",20.8,1.1,0.047120,0.052885,31,29.6,25.9,25.2
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",28.9,3.1,"Brigantine city, Atlantic County",23.5,2.5,0.107266,0.106383,31,29.6,25.9,25.2
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",27.7,3.2,"Buena borough, Atlantic County",28.9,3.3,0.115523,0.114187,31,29.6,25.9,25.2


In [86]:
muni_merge['time_change'] = ((pd.to_numeric(muni_merge['2015meantime'], errors='coerce') - pd.to_numeric(muni_merge['2009meantime'], errors="coerce"))/pd.to_numeric(muni_merge['2009meantime'], errors="coerce"))*100    
muni_merge['nj_time_change'] = ((pd.to_numeric(muni_merge['nj_latest'], errors='coerce') - muni_merge['nj_previous'])/muni_merge['nj_previous'])*100
muni_merge['us_time_change'] = ((muni_merge['us_latest']-muni_merge['us_previous'])/muni_merge['us_previous'])*100
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015meantime,2015meanmoe,GEOdisplaylabel,2009meantime,2009meanmoe,2015meanmoeper,2009meanmoeper,nj_latest,nj_previous,us_latest,us_previous,time_change,nj_time_change,us_time_change
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",N,N,NaN,NaN,31,29.6,25.9,25.2,NaN,4.72973,2.777778
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",22.7,2.4,"Absecon city, Atlantic County",19.3,1.6,0.105727,0.082902,31,29.6,25.9,25.2,17.616580,4.72973,2.777778
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",19.1,0.9,"Atlantic City city, Atlantic County",20.8,1.1,0.047120,0.052885,31,29.6,25.9,25.2,-8.173077,4.72973,2.777778
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",28.9,3.1,"Brigantine city, Atlantic County",23.5,2.5,0.107266,0.106383,31,29.6,25.9,25.2,22.978723,4.72973,2.777778
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",27.7,3.2,"Buena borough, Atlantic County",28.9,3.3,0.115523,0.114187,31,29.6,25.9,25.2,-4.152249,4.72973,2.777778


In [204]:
muni_merge.head()

,GEOdisplaylabel,County,GEO.id,GEO.id2,GEOdisplaylabel_long,2015value,2015valuemoe,GEOdisplaylabel,2009valuewoinf,2009valuemoewoinf,...,2009_median_value_moe,nj_latest,nj_previous,us_latest,us_previous,2015valuemoeper,2009valuemoeper,value_change,nj_value_change,us_value_change
0,County subdivisions not defined,Atlantic,0600000US3400100000,3400100000,"County subdivisions not defined, Atlantic Coun...",-,**,"County subdivisions not defined, Atlantic County",NaN,NaN,...,NaN,315900,392647.553994,178600,204027.064211,NaN,NaN,NaN,-19.546169,-12.462594
1,Absecon city,Atlantic,0600000US3400100100,3400100100,"Absecon city, Atlantic County, New Jersey",206900,16458,"Absecon city, Atlantic County",259100.0,17220.0,...,18950.086546,315900,392647.553994,178600,204027.064211,0.079546,0.066461,-27.437034,-19.546169,-12.462594
2,Atlantic City city,Atlantic,0600000US3400102080,3400102080,"Atlantic City city, Atlantic County, New Jersey",198200,15644,"Atlantic City city, Atlantic County",231100.0,10782.0,...,11865.263249,315900,392647.553994,178600,204027.064211,0.078930,0.046655,-22.066236,-19.546169,-12.462594
3,Brigantine city,Atlantic,0600000US3400107810,3400107810,"Brigantine city, Atlantic County, New Jersey",364000,15718,"Brigantine city, Atlantic County",448200.0,18508.0,...,20367.491394,315900,392647.553994,178600,204027.064211,0.043181,0.041294,-26.200829,-19.546169,-12.462594
4,Buena borough,Atlantic,0600000US3400108680,3400108680,"Buena borough, Atlantic County, New Jersey",187500,10554,"Buena borough, Atlantic County",199800.0,25289.0,...,27829.775765,315900,392647.553994,178600,204027.064211,0.056288,0.126572,-14.723820,-19.546169,-12.462594


In [87]:
muni_merge.to_csv("commutetimes.csv")